# Self study 2


In this self-study we build an index that supports Boolean search over the web pages that you crawl with the crawler from the 1st self study. You can continue to just extract the titles of the web-pages you crawl, or you can be more adventurous and look at the whole text that you get from the .get_text() method of a BeautifulSoup parser. In either case, the collection of texts from the crawled web-pages is you corpus. You should then:

- construct the vocabulary of terms for your corpus
- build an 'inverted' index for your vocabulary
- implement Boolean search for your index (perhaps only for a limited set of Boolean queries)

In [21]:
# Some things already used in self study 1:
import requests
from bs4 import BeautifulSoup
from time import sleep
from urllib.robotparser import RobotFileParser
from urllib.parse import urlparse
from datetime import datetime, timedelta
from itertools import count
import pandas as pd
import numpy as np
import random
import nltk
nltk.download('punkt')
from nltk.stem.snowball import SnowballStemmer
import codecs


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\johan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


What is most useful for you depends on which websites you crawl. It is not essential for the exercise that the stemming always is the best possible ...!

In [22]:
index_frame = pd.read_csv('index.csv')
index_arr = index_frame.to_dict('records')

index_arr = index_arr[0]
index_arr.pop('Unnamed: 0')

for key in index_arr.keys():
    print(f"{key}: {index_arr[key]}")

https://www.tv2.dk: TV 2 - bedst pÃ¥ breaking og live
https://www.bt.dk: B.T. Nyheder | Læs nyhederne på bt.dk
https://www.berlingskemedia.dk/handelsbetingelser: Generelle handelsbetingelser
https://www.bt.dk/cookiedeklaration: Cookiedeklaration
https://www.dr.dk: DR | Nyheder - Breaking - TV - Radio
https://www.bt.dk/kongehusetbagkulissen: Kongehuset bag kulissen
https://www.weekendavisen.dk/mine-sider/kundeservice: Mine sider | Weekendavisen
https://www.weekendavisen.dk/: Weekendavisen | Weekendavisen
https://www.berlingske.dk/mine-sider/kundeservice: Nyheder og seneste nyt fra Berlingske - Berlingske.dk
https://www.weekendavisen.dk/titusinde: Podcast: Lone Frank & de titusinde sjæle | Weekendavisen
https://www.weekendavisen.dk/en-ny-ladning-roeverhistorier: En ny ladning Røverhistorier | Weekendavisen
https://www.bt.dk/mine-sider/kundeservice: BT | Mine sider
https://www.dr.dk/om-dr/vilkaar-paa-drdk: Vilkår på dr.dk | Om DR | DR
https://www.dr.dk/om-dr/job: Job | DR
https://www.euro

In [23]:
for key in index_arr.keys():
    print(f"{key}: {index_arr[key]}")

https://www.tv2.dk: TV 2 - bedst pÃ¥ breaking og live
https://www.bt.dk: B.T. Nyheder | Læs nyhederne på bt.dk
https://www.berlingskemedia.dk/handelsbetingelser: Generelle handelsbetingelser
https://www.bt.dk/cookiedeklaration: Cookiedeklaration
https://www.dr.dk: DR | Nyheder - Breaking - TV - Radio
https://www.bt.dk/kongehusetbagkulissen: Kongehuset bag kulissen
https://www.weekendavisen.dk/mine-sider/kundeservice: Mine sider | Weekendavisen
https://www.weekendavisen.dk/: Weekendavisen | Weekendavisen
https://www.berlingske.dk/mine-sider/kundeservice: Nyheder og seneste nyt fra Berlingske - Berlingske.dk
https://www.weekendavisen.dk/titusinde: Podcast: Lone Frank & de titusinde sjæle | Weekendavisen
https://www.weekendavisen.dk/en-ny-ladning-roeverhistorier: En ny ladning Røverhistorier | Weekendavisen
https://www.bt.dk/mine-sider/kundeservice: BT | Mine sider
https://www.dr.dk/om-dr/vilkaar-paa-drdk: Vilkår på dr.dk | Om DR | DR
https://www.dr.dk/om-dr/job: Job | DR
https://www.euro

In [50]:
dstemmer=SnowballStemmer("danish")
a_file = codecs.open("stopord.txt", "r", "utf-8")
file_contents = a_file.read()
contents_split = file_contents.splitlines()
stop_words = []
for word in contents_split:
    if dstemmer.stem(word) not in stop_words:
        stop_words.append(dstemmer.stem(word))
extra_words = ["|", ",", ".", "-", "!"]
for word in extra_words:
    stop_words.append(word)

for word in stop_words:
    print(word)
a_file.close()

ad
af
aldr
alen
all
alligevel
alt
altid
and
andr
at
bag
bar
beg
bl.a.
bland
blev
bliv
burd
bør
ca.
da
de
dem
den
der
dereft
derfor
derfra
deri
dermed
derpå
derved
det
dig
din
dis
dit
dog
du
eft
egen
ej
ell
en
end
endnu
ene
enest
enhv
ens
ent
er
et
f.eks.
far
fem
fik
fir
fler
flest
for
foran
fordi
fra
fx
få
får
før
først
gennem
gjord
gjort
god
godt
gør
ham
han
har
havd
hav
hej
hel
helt
hen
hend
henov
her
hereft
heri
hermed
herpå
hos
hun
hvad
hvem
hver
hvilk
hvis
hvor
hvordan
hvoreft
hvorfor
hvorfra
hvorh
hvori
hvorimod
hvornår
hvorved
i
igen
igennem
ikk
imellem
imen
imod
ind
indtil
ing
int
ja
jeg
jer
jo
kan
kom
kun
lad
lang
lav
lidt
lig
ligesom
lil
læng
man
mand
mang
med
meg
mellem
men
mer
mest
mig
min
mindr
mindst
mit
mod
må
måsk
ned
nej
nem
ni
nog
nogensind
nogl
nok
nu
ny
nyt
når
nær
næst
og
også
okay
om
omkring
op
os
ott
over
overalt
pga.
på
sam
se
sek
selv
selvom
sen
ser
ses
sid
sig
sin
sit
skal
skul
som
stad
stor
syn
synt
syv
så
sådan
såled
tag
tem
thi
ti
tid
til
tilbag
tit
to
tre


In [55]:
stemmed_index = {}
for key in index_arr.keys():
    try:
        tokens = nltk.word_tokenize(index_arr[key])
        local_tokens = []
        for token in tokens:
            if token not in stop_words:
                local_tokens.append(dstemmer.stem(token))
        stemmed_index[key] = local_tokens
    except:
        print(f'error with:  {key}')
        continue

#used to give numbers
numbered_index = {}
i = 1
for url in stemmed_index.keys():
    stemmed_index[url] = {'tokens' : stemmed_index[url], 'id' : i}
    numbered_index[i] = url
    i += 1
for key in numbered_index.keys():
    print(f'{key}: {numbered_index[key]}')

error with:  https://www.nyheder.aau.dk/2020
error with:  https://www.sundhed.dk/borger/guides/sundhedstilbud/?SearchTerm=&searchType=searchTerm&Page=1&PageSize=10&Tilbudstype=-1&Region=-1&Udbyder=-1
error with:  https://www.aub.aau.dk/find-materiale/databaser/info?id=1352281
error with:  https://www.nordiskemediermedlemskab.dk
1: https://www.tv2.dk
2: https://www.bt.dk
3: https://www.berlingskemedia.dk/handelsbetingelser
4: https://www.bt.dk/cookiedeklaration
5: https://www.dr.dk
6: https://www.bt.dk/kongehusetbagkulissen
7: https://www.weekendavisen.dk/mine-sider/kundeservice
8: https://www.weekendavisen.dk/
9: https://www.berlingske.dk/mine-sider/kundeservice
10: https://www.weekendavisen.dk/titusinde
11: https://www.weekendavisen.dk/en-ny-ladning-roeverhistorier
12: https://www.bt.dk/mine-sider/kundeservice
13: https://www.dr.dk/om-dr/vilkaar-paa-drdk
14: https://www.dr.dk/om-dr/job
15: https://www.euroinvestor.dk/
16: https://www.pressenaevnet.dk/
17: https://www.dr.dk/nyheder/nyh

In [56]:
for key in stemmed_index.keys():
    print(f"{key}: {stemmed_index[key]['tokens']} [{stemmed_index[key]['id']}]")

https://www.tv2.dk: ['tv', '2', 'bedst', 'pã¥', 'breaking', 'liv'] [1]
https://www.bt.dk: ['b.t', 'nyhed', 'læs', 'nyhed', 'bt.dk'] [2]
https://www.berlingskemedia.dk/handelsbetingelser: ['generel', 'handelsbeting'] [3]
https://www.bt.dk/cookiedeklaration: ['cookiedeklaration'] [4]
https://www.dr.dk: ['dr', 'nyhed', 'breaking', 'tv', 'radio'] [5]
https://www.bt.dk/kongehusetbagkulissen: ['kongehus', 'kulis'] [6]
https://www.weekendavisen.dk/mine-sider/kundeservice: ['min', 'sid', 'weekendavis'] [7]
https://www.weekendavisen.dk/: ['weekendavis', 'weekendavis'] [8]
https://www.berlingske.dk/mine-sider/kundeservice: ['nyhed', 'senest', 'berlingsk', 'berlingske.dk'] [9]
https://www.weekendavisen.dk/titusinde: ['podcast', ':', 'lon', 'frank', '&', 'titusind', 'sjæl', 'weekendavis'] [10]
https://www.weekendavisen.dk/en-ny-ladning-roeverhistorier: ['en', 'ladning', 'røverhistori', 'weekendavis'] [11]
https://www.bt.dk/mine-sider/kundeservice: ['bt', 'min', 'sid'] [12]
https://www.dr.dk/om-dr/

In [57]:
simple_inverted_index = {}

for url in stemmed_index.keys():
    for token in stemmed_index[url]['tokens']:
        if token in simple_inverted_index.keys():
            if stemmed_index[url]['id'] not in simple_inverted_index[token]:
                simple_inverted_index[token].append(stemmed_index[url]['id'])
        else:
            simple_inverted_index[token] = [stemmed_index[url]['id']]

for key in simple_inverted_index.keys():
    print(f"{key}: {simple_inverted_index[key]}")
    

tv: [1, 5, 861]
2: [1]
bedst: [1, 829]
pã¥: [1]
breaking: [1, 5]
liv: [1, 579]
b.t: [2]
nyhed: [2, 5, 9, 81, 148, 199, 224, 488, 543, 549, 593, 707, 743, 802, 804, 808, 885]
læs: [2, 148, 538]
bt.dk: [2]
generel: [3]
handelsbeting: [3]
cookiedeklaration: [4, 93]
dr: [5, 13, 14, 17, 20, 27, 81, 242, 246]
radio: [5]
kongehus: [6, 459]
kulis: [6]
min: [7, 12]
sid: [7, 12, 72, 97, 196, 255, 491, 506, 598, 742, 783, 796, 909, 911]
weekendavis: [7, 8, 10, 11, 18, 22, 25, 26, 56, 68, 145, 188, 361, 389, 396, 554, 641, 712, 857, 894]
senest: [9, 224]
berlingsk: [9, 148, 224]
berlingske.dk: [9, 148, 224]
podcast: [10, 21, 65, 78]
:: [10, 16, 20, 27, 47, 51, 194, 204, 237, 278, 286, 342, 458, 479, 480, 515, 518, 681, 698, 715, 794, 833, 836, 849, 851, 852, 901]
lon: [10]
frank: [10]
&: [10, 251, 289, 574, 588, 622, 679, 684, 691, 751, 815, 899, 900, 907, 921]
titusind: [10]
sjæl: [10]
en: [11, 68, 836]
ladning: [11]
røverhistori: [11, 22]
bt: [12, 47, 51, 194, 204, 237, 247, 464, 479, 480, 515, 

In [59]:
#Single word search

def convert_postings_to_urls(found_postings):
    found_urls = []
    for posting in found_postings:
        found_urls.append(numbered_index[posting])
    return found_urls

def single_word_search(search_word, want_postings = False):
    found_postings = []
    for token in simple_inverted_index.keys():
        if (token == search_word):
            found_postings = simple_inverted_index[token]
    if want_postings:
        return found_postings
    if found_postings:
        return convert_postings_to_urls(found_postings)
    return []
    

for url in single_word_search('min'):
    print(url)

https://www.weekendavisen.dk/mine-sider/kundeservice
https://www.bt.dk/mine-sider/kundeservice


In [61]:
#Search

def OR_merge(found_urls_one, found_urls_two):
    return list(set(found_urls_one + found_urls_two))

def AND_merge(found_urls_one, found_urls_two):
    return list(set(found_urls_one).intersection(set(found_urls_two)))

def AND_NOT_merge(found_urls_one, found_urls_two):
    a = set(found_urls_one)
    b = set(found_urls_two)    
    return list(a-b)

def Search(word_one, search_type, word_two):
    stemmed_word_one = dstemmer.stem(word_one)
    stemmed_word_two = dstemmer.stem(word_two)
    found_urls_one = single_word_search(stemmed_word_one, want_postings=True)
    found_urls_two = single_word_search(stemmed_word_two, want_postings=True)
    if(search_type == 'AND'):
        return AND_merge(found_urls_one, found_urls_two)
    elif(search_type == 'OR'):
        return OR_merge(found_urls_one, found_urls_two)
    elif(search_type == 'AND NOT'):
        return AND_NOT_merge(found_urls_one, found_urls_two)
    else:
        return ['Invalid search type']

for url in convert_postings_to_urls(Search('Universitet', 'OR', 'Kat')):
    print(url)

SyntaxError: unmatched ')' (3432903619.py, line 20)